In [1]:
from typing import Literal

from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph, MessagesState 
from langgraph.prebuilt import ToolNode
from langgraph.types import Command 



In [2]:
@tool
def search(query: str):
    """Call to surt the web."""
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny"

tools = [search]
tool_node = ToolNode(tools)

In [3]:
model = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools)

In [4]:
def call_model(state: MessagesState) -> Command[Literal['tools', END]]:
    messages = state['messages']
    response = model.invoke(messages)
    if len(response.tool_calls) > 0:
        next_node = "tools"
    else:
        next_node = END
    return Command(goto=next_node, update={"messages": [response]})

In [5]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_edge("tools", "agent")
app = workflow.compile()

In [6]:
response = app.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in sf"}]},
    debug=True
)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': 'What is the weather in sf', 'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': 'What is the weather in sf', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='What is the weather in sf', additional_kwargs={}, response_metadata={}, id='e3736aa4-7d8a-4e6b-b91e-bdd8e01fb5be')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'messages': [HumanMessage(content='What is the weather in sf', additional_kwargs={}, response_metadata={}, id='e3736aa4-7d8a-4e6b-b91e-bdd8e01fb5be')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7ZyIiOfmU8a6MzOICkrNXCAb', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'search'}, 'type': 'function'}], 'r

In [7]:
def transfer_to_weather_agent():
    """Call this to transfer to the weather agent."""

tools = [transfer_to_weather_agent]
main_model = ChatOpenAI(model="gpt-4", temperature=0).bind_tools(tools)

In [ ]:
def call_model(state: MessagesState) -> Command[Literal['weather_agent', END]]:
    messages = state['messages']
    response = main_model.invoke(messages)
    if len(response.tool_calls) > 0:
        return Command(goto="weather_agent")
    else:
        return Command(goto=END, update={"messages": [response]})